In [12]:
from d2l import torch as d2l
from torch import nn
import torch

batch_size = 256
# 调用封装好的加载数据集的函数
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

Fashion-MNIST是一个替代MNIST手写数字集的图像数据集。它是由Zalando（一家德国的时尚科技公司）旗下的研究部门提供的。该数据集包含了10个类别的共7万张灰度图像，涵盖了从衣物到鞋子等不同种类的时尚物品。每个类别有7000张图像，其中6000张是训练图像，1000张是测试图像。每张图片的大小为28x28像素12。

在PyTorch中，使用torchvision.datasets.FashionMNIST可以方便地读取Fashion-MNIST数据集。如果您使用PyTorch读取Fashion-MNIST数据集，您会发现训练集的tensor shape为[60000, 1, 28, 28]，而测试集的tensor shape为[10000, 1, 28, 28]。因此，您读取到的训练集tensor shape为[256, 1, 28, 28]可能是由于您在读取数据时使用了batch_size=256的参数设置。31

In [13]:
# 分析下数据集
for X, y in train_iter:
    print(type(X)), print(type(y))
    print(X.shape), print(y.shape)
    # y 为实际的类型
    print(y[range(10)])
    break

<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([256, 1, 28, 28])
torch.Size([256])
tensor([6, 5, 3, 0, 1, 1, 7, 0, 0, 3])


创建单层的神经网络  
将 softmax 作为该层神经网络的激活函数

> softmax和sigmoid函数都是神经网络中常用的激活函数。在神经网络中，激活函数通常被放置在每个神经元的输出端，用>于将神经元的输出转换为非线性形式。这样可以使神经网络能够学习非线性关系，从而提高模型的表达能力1。  
> sigmoid函数是一种常用的激活函数，它将输入值映射到0到1之间的值。在二分类问题中，sigmoid函数通常被用于输出层，将输出值转换为0到1之间的概率值。在多标签分类问题中，sigmoid函数也可以被用于输出层，将输出值转换为0到1之间的概率值2.  
> softmax函数是一种常用的激活函数，它将输入值映射到0到1之间的值，并且所有输出值的和为1。在多分类问题中，softmax函数通常被用于输出层，将输出值转换为0到1之间的概率值，并且所有输出概率之和为13.  
因此，在神经网络中，sigmoid函数通常被用于二分类或多标签分类问题中，而softmax函数通常被用于多分类问题中。

在 PyTorch 中，nn.CrossEntropyLoss() 函数是用于计算交叉熵损失的函数。其中 reduction 参数用于控制输出损失的形式。当 reduction=‘none’ 时，函数会输出一个形状为 (batch_size, num_classes) 的矩阵，表示每个样本的每个类别的损失1。

换句话说，当 reduction=‘none’ 时，函数不会对每个样本的损失进行求和或平均，而是返回一个形状为 (batch_size, num_classes) 的矩阵，其中每个元素表示该样本在该类别上的损失值1。

$$
    z = <\vec X, \vec W> + b
$$


$$\theta_{t+1} = \theta_t - \eta \nabla f(\theta_t)$$
> 其中，θt​ 表示第 t 次迭代的参数，η 表示学习率，∇f(θt​) 表示损失函数在 θt​ 处的梯度


$$
\mathrm{softmax}(\mathbf{X})_{ij} = \frac{\exp(\mathbf{X}_{ij})}{\sum_k \exp(\mathbf{X}_{ik})}.
$$

##### SGD  
SGD 是随机梯度下降（stochastic gradient descent）的缩写。它是一种常用的优化算法，用于训练神经网络模型。在每次迭代中，SGD 会随机选择一部分样本来计算梯度，并使用这些梯度来更新模型的参数。由于每次迭代中只使用了一部分样本，因此 SGD 的计算速度比全批量梯度下降（batch gradient descent）快得多。但是，由于 SGD 只使用了一部分样本，因此它的梯度估计可能会存在噪声，从而导致模型训练过程中出现震荡1。

##### net parameters  
在 PyTorch 中，net.parameters() 是一个函数，它返回一个包含模型所有参数的迭代器。这些参数可以用于优化器的更新操作，例如 SGD、Adam 等

In [14]:
# 添加一层 输入向量 784 个参数 输出向量 10 个类别
neural_network = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

# 初始化权重
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

neural_network.apply(init_weights);

# 梯度下降的优化算法
gradient_decent = torch.optim.SGD(neural_network.parameters(), lr=0.1)

print(neural_network.parameters())

<generator object Module.parameters at 0x000001CBE82EAAC0>


[重新审视 Softmax 的实现](https://zh-v2.d2l.ai/chapter_linear-networks/softmax-regression-concise.html#id3)

当我们使用softmax函数时，我们通常会将一个向量作为输入。例如，假设我们有一个大小为4的向量[1.0, 2.0, 3.0, 4.0]，我们想将其转换为概率分布。我们可以使用softmax函数来实现这一点。softmax函数的输出是一个概率分布，其中每个元素都是非负的，并且所有元素的和为1。在这个例子中，softmax函数的输出将是[0.0321, 0.0871, 0.2369, 0.6439]。

在计算softmax时，如果向量中的一些数值非常大，那么exp(向量中的数值)可能大于数据类型容许的最大数字，即上溢（overflow）。这将使分母或分子变为inf（无穷大），最后得到的是0、inf或nan（不是数字）的概率分布。为了解决这个问题，我们可以在继续softmax计算之前，先从所有向量中减去max(向量)。在这个例子中，max([1.0, 2.0, 3.0, 4.0]) = 4.0。因此，在计算softmax之前，我们将每个元素减去4.0。这样做不会改变softmax的返回值，但可以避免上溢。在减法和规范化步骤之后，可能有些元素具有较大的负值。由于精度受限，exp(这些负值)将有接近零的值，即下溢（underflow）。这些值可能会四舍五入为零，使得概率分布为零，并且使得log(概率分布)的值为-inf。

In [15]:
# 交叉熵损失函数 -> 
# 注意使用 reduction='none' 来使 log 和 softmax 的 e 进行抵消
loss = nn.CrossEntropyLoss(reduction='none')

在PyTorch中，neural_network.train()是用来将神经网络模型设置为训练模式的。在训练模式下，神经网络会更新权重和偏差，以便更好地拟合训练数据。此外，在训练模式下，一些层（如Dropout和BatchNorm）会表现出不同的行为。例如，在训练模式下，BatchNorm会在每个新批次上更新移动平均值；而在评估模式下，这些更新会被冻结。1

当你完成训练时，你可以使用neural_network.eval()将神经网络模型设置为评估模式。在评估模式下，神经网络不会更新权重和偏差，并且所有层都会表现出相同的行为。

In [16]:
neural_network.train()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
)

In [38]:
for i in range(100):
    collector = torch.empty(0)
    for X, y in train_iter:
        # 计算损失
        y_hat = neural_network(X)
        _, y_predict = torch.max(y_hat, dim=1) 
        right_counter = 0
        for i in range(y_predict.shape[0]):
            if y_predict[i].item() == y[i].item():
                right_counter += 1
        collector = torch.cat((collector, torch.tensor([right_counter / y_predict.shape[0]])))
        l = loss(y_hat, y)
        # 梯度清零
        gradient_decent.zero_grad()
        # 反向传播
        lm = l.mean()
        # print(lm)
        lm.backward()
        # 更新参数
        gradient_decent.step()
    print(f'平均预测率 -> {collector.mean()}')
# for name, param in neural_network.named_parameters():
#     if param.requires_grad:
#         if name == '1.bias':
#             print(name, param.data)

最终平均预测率 -> 0.8533188700675964
最终平均预测率 -> 0.8533521294593811
最终平均预测率 -> 0.8552250266075134
最终平均预测率 -> 0.854327380657196
最终平均预测率 -> 0.8561502695083618
最终平均预测率 -> 0.8564550280570984
最终平均预测率 -> 0.8577404022216797
最终平均预测率 -> 0.8570090532302856
最终平均预测率 -> 0.8576020002365112
最终平均预测率 -> 0.8587377667427063
最终平均预测率 -> 0.8593916296958923
最终平均预测率 -> 0.8580784797668457
最终平均预测率 -> 0.8599511981010437
最终平均预测率 -> 0.8605053424835205
最终平均预测率 -> 0.8601452112197876
最终平均预测率 -> 0.8594969511032104
最终平均预测率 -> 0.8606881499290466
最终平均预测率 -> 0.8611259460449219
最终平均预测率 -> 0.861358642578125
最终平均预测率 -> 0.8618128895759583
最终平均预测率 -> 0.8623005151748657
最终平均预测率 -> 0.8623393774032593
最终平均预测率 -> 0.8624224662780762
最终平均预测率 -> 0.8622395992279053
最终平均预测率 -> 0.8627936244010925
最终平均预测率 -> 0.8620401620864868
最终平均预测率 -> 0.8626053333282471
最终平均预测率 -> 0.8626219630241394
最终平均预测率 -> 0.8629986643791199
最终平均预测率 -> 0.8636912107467651
最终平均预测率 -> 0.8631260395050049
最终平均预测率 -> 0.8643174171447754
最终平均预测率 -> 0.8639517426490784
最终平均预测率 -> 0

KeyboardInterrupt: 